In [11]:
from flask import Flask, request
from flask import render_template
from werkzeug.utils import secure_filename
import time
from flask_cors import CORS
from flask_httpauth import HTTPBasicAuth
import sqlite3
import pandas as pd 
import bcrypt
import requests

In [12]:
def dbConnection():
    con = sqlite3.connect("data.db")
    return con

In [13]:
def getSqlEntry(filter_col,filter_data,result_col,table="users"):
    with dbConnection() as con:
        cur = con.execute(f"SELECT {result_col} FROM {table} WHERE {filter_col}=?",(filter_data,))
        hashed_password = cur.fetchone()
        if hashed_password:
            return(hashed_password[0])
        else:
            print("Data not found")
            return None

In [14]:
saved_password = getSqlEntry("Username","matt","Password")
print(saved_password)
type(saved_password)

b'$2b$12$Rm20GcQH8MsFBQXDQMkzDecCQj9IPswipGtD3NB.ZrZNuArcuxIh.'


bytes

In [15]:
with dbConnection() as con:
    login_username = "test2"
    cur = con.execute("SELECT Password FROM users WHERE Username=?",(login_username,))
    hashed_password = cur.fetchone()
    print(hashed_password[0])
    # if hashed_password:
    #     password = hashed_password[0]
    #     print(password)

TypeError: 'NoneType' object is not subscriptable

In [16]:
def get_weather(lat,long):
    url = "https://api.open-meteo.com/v1/forecast"
    parameters = {
        "latitude":lat,
        "longitude":long,
        "current": "temperature_2m",
        "temperature_unit": "fahrenheit",
	    "timezone": "GMT"
    }
    res = requests.get(url, params=parameters)
    data = res.json()
    return data

In [17]:
lat = "33.9410944"
long="-83.3486848"
weather = get_weather(lat,long)
weather['current']

{'time': '2024-04-25T15:30', 'interval': 900, 'temperature_2m': 71.2}

In [18]:
get_weather(lat,long)['current']['time']

'2024-04-25T15:30'

In [19]:
weather_api_key = 'bf48bd240e344b0082423017242504'
url = 'http://api.weatherapi.com/v1'
parameters = {
    "key":weather_api_key,
    "q":"33.9410944,-83.3486848"
}
res = requests.get(url, params=parameters)
data = res.json()
data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
def get_location(lat,long):
    locate_url = f'https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={long}&format=json'
    res = requests.get(locate_url).json()
    data = [res['address']['state'], res['address']['country']]
    return data



In [26]:
lat = "33.9410944"
long="-83.3486848"
location = get_location(lat,long)
print(location)

['Georgia', 'United States']


In [42]:
def get_seniment(description):
    prompt = f"Is the sentiment of this description bad, neutral, or good? Return the result in a one-word answer: {description}"
    host = "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent"
    apikey = open("gemini_apikey.txt").read() # Need to direct to your own api key
    parameter = {"key":apikey}
    body = {"contents":[{"parts":[{"text":f'{prompt}'}]}]}
    res = requests.post(host,params=parameter,json=body)
    res.json()
    gem_res = res.json()['candidates'][0]['content']['parts'][0]['text']
    return gem_res

In [43]:
description = "this is a picture of my enemy joe"
get_seniment(description)

'Bad'

In [44]:
get_weather(lat,long)['current']['time']

'2024-04-25T17:30'

In [46]:
import os

def get_valid_filepath(f):
    if f:
        file_name = secure_filename(f.filename)
        if os.path.exists('./uploads/{file_name}'):
            # If the file exists, generate a new name by appending a number to the filename
            name, extension = os.path.splitext(file_name)
            count = 1
            new_filename = f"{name}({count}){extension}"
            while os.path.exists('./uploads/{new_filename}'):
                count += 1
                new_filename = f"{name}_{count}{extension}"
                file_name = new_filename
        file_path = f"./uploads/{file_name}"
    else:
        file_path = "No File Uploaded"
    return file_path




IndentationError: unindent does not match any outer indentation level (<string>, line 9)

In [ ]:
        f.save(file_path) 
